In [ ]:
    """
    Workflow
    1. Generate 2 test entities for this
        - The pooler bank (us)
        - The user itself
    2. Generate a test transaction
    3. Link up with stripe for said transaction
    """

# Plaid Endpoint

In [1]:

%load_ext dotenv
%dotenv

import plaid
import os
from plaid.api import plaid_api

# Available environments are
# 'Production'
# 'Development'
# 'Sandbox'
configuration = plaid.Configuration(
    host=plaid.Environment.Sandbox,
    api_key={
        'clientId': os.getenv('PLAID_CLIENT_ID'),
        'secret': os.getenv('PLAID_SANDBOX_KEY'),
    }
)

api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)


## Get Institutions by Id

In [46]:
from plaid.model.institutions_get_request import InstitutionsGetRequest
from plaid.model.institutions_get_request_options import InstitutionsGetRequestOptions
from plaid.model.country_code import CountryCode    # CountryCode.US

product_list = ['payment_initiation', 'balance', 'assets', 'transactions']
country_codes = ['US', 'GB', 'ES']
request = InstitutionsGetRequest(
    country_codes=[CountryCode(cc) for cc in country_codes],
    count = 1,
    offset= 0,
    options = InstitutionsGetRequestOptions(oauth=True, products=[Products(p) for p in product_list])
)

response = client.institutions_get(request)
institutions = response['institutions']
institutions

[{'country_codes': ['GB'],
  'institution_id': 'ins_127298',
  'name': 'C. Hoare & Co',
  'oauth': True,
  'products': ['assets',
               'auth',
               'balance',
               'transactions',
               'identity',
               'payment_initiation',
               'standing_orders'],
  'routing_numbers': []}]

## Create dummy item

In [2]:
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
from plaid.model.products import Products
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest

In [4]:
pt_request = SandboxPublicTokenCreateRequest(
    institution_id="ins_127298",
    initial_products=[Products('transactions')]
)
pt_response = client.sandbox_public_token_create(pt_request)
print(pt_response)

# The generated public_token can now be
# exchanged for an access_token

# exchange_request = ItemPublicTokenExchangeRequest(
#     public_token=pt_response['public_token']
# )
# exchange_response = client.item_public_token_exchange(exchange_request)
# print(exchange_response)

## Query for account details

In [53]:
from plaid.model.accounts_get_request import AccountsGetRequest

ACCESS_TOKEN_1 = 'access-sandbox-55938299-802e-419a-a37d-2d7805166c4b'
ACCESS_TOKEN_2 = 'access-sandbox-5f744d07-7804-4254-881c-a239f3c160f0'
request = AccountsGetRequest(access_token=ACCESS_TOKEN_1)
response = client.accounts_get(request)
accounts = response['accounts']
accounts

[{'account_id': 'NvWAzBlRA9h9kyEJk83Gtw8l1K7rd6irlzady',
  'balances': {'available': 100.0,
               'current': 110.0,
               'iso_currency_code': 'GBP',
               'limit': None,
               'unofficial_currency_code': None},
  'mask': '0000',
  'name': 'Plaid Checking',
  'official_name': 'Plaid Gold Standard 0% Interest Checking',
  'subtype': 'checking',
  'type': 'depository'},
 {'account_id': 'PvGm3xKbmwh6xWpmxLA1ugA3yqlzJVswxnjaX',
  'balances': {'available': 200.0,
               'current': 210.0,
               'iso_currency_code': 'GBP',
               'limit': None,
               'unofficial_currency_code': None},
  'mask': '1111',
  'name': 'Plaid Saving',
  'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
  'subtype': 'savings',
  'type': 'depository'},
 {'account_id': 'jBowy9GbwahQzw1nzR6rfDlK5BydX3CyNj9dZ',
  'balances': {'available': None,
               'current': 1000.0,
               'iso_currency_code': 'GBP',
               'lim

## Transfer money to another client

In [54]:

from plaid.model.bank_transfer_create_request import BankTransferCreateRequest
from plaid.model.bank_transfer_type import BankTransferType
from plaid.model.bank_transfer_network import BankTransferNetwork
from plaid.model.bank_transfer_user import BankTransferUser
from plaid.model.ach_class import ACHClass

request = BankTransferCreateRequest(
    idempotency_key='random string',
    access_token= ACCESS_TOKEN_2, # user unqiue, source
    account_id='NvWAzBlRA9h9kyEJk83Gtw8l1K7rd6irlzady', # target
    type=BankTransferType('credit'),
    network=BankTransferNetwork('ach'),
    amount='1.00',
    iso_currency_code='USD',
    description='payment',
    user=BankTransferUser(legal_name='Firstname Lastname'),
    metadata=None,
    ach_class=ACHClass('ppd'),
    custom_tag='',
    origination_account_id = '11111111-1111-1111-1111-111111111111',
)
response = client.bank_transfer_create(request)
bank_transfer = response['bank_transfer']

NameError: name 'ACHClass' is not defined